In [7]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from unicodedata import normalize

## Reading file from the wiki pedia Assignment 1
##   1. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
##    2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
## More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.
##    3.If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
##    4. Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
##    5. In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [6]:
table_CAN = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df_TN = table_CAN[0]
df_TN['Borough'].replace("Not assigned", np.nan, inplace = True)
df_TN.dropna(subset=["Borough"], axis=0, inplace=True)
# reset index, because we droped two rows
df_TN.reset_index(drop=True, inplace=True)
df_TN['Neighbourhood'].replace("Not assigned", df_TN['Borough'], inplace = True)

In [ ]:
df_TN.shape